In [ ]:
'''
I have 3 datasets: 
1. manual_label_batch1.xlsx -> we are gonna convert all other tags label 2-9 as label 1 to put them into relevant catagory
and save the file as csv file "x". this file has 177 data rows
2. burnout/-search_results.csv -> check if this file contains same data as file "x", if so, then we want to merge the remaining 
data rows of this file to the "x" to label the remaning data. this file has 289 rows

both files has different data frame header

'''

In [1]:
import pandas as pd

# --- Load files ---
df1 = pd.read_excel('manual_label_batch1.xlsx')
df2 = pd.read_csv('burnout_search_results.csv')

In [2]:
# --- Step 1: Remap labels 2-9 and compound labels (e.g. '7,8', '1,4') → 1, keep 0 as 0 ---
def remap_label(val):
    if str(val) == '0':
        return 0
    return 1

df1['label'] = df1['label'].apply(remap_label)

print('Label distribution after remap:')
print(df1['label'].value_counts().sort_index())
print(f'Total rows in df1: {len(df1)}')


Label distribution after remap:
label
0    107
1     70
Name: count, dtype: int64
Total rows in df1: 177


In [3]:
# --- Step 2: Check overlap between df1 and df2 by 'id' ---
ids_in_df1 = set(df1['id'].astype(str))
df2['id'] = df2['id'].astype(str)

overlap   = df2[df2['id'].isin(ids_in_df1)]
remaining = df2[~df2['id'].isin(ids_in_df1)]

print(f'\nOverlapping rows (df2 already in df1): {len(overlap)}')
print(f'Remaining rows to label from df2:     {len(remaining)}')



Overlapping rows (df2 already in df1): 177
Remaining rows to label from df2:     112


In [4]:
# --- Step 3: Format remaining rows to match df1 schema ---
remaining_formatted = pd.DataFrame({
    'id':               remaining['id'],
    'text':             remaining['text'],
    'similarity_score': remaining['similarity_score'],
    'label':            None   # unlabeled — awaiting annotation
})

In [5]:
# --- Step 4: Merge labeled df1 + unlabeled remaining rows ---
merged = pd.concat([df1, remaining_formatted], ignore_index=True)

print(f'\nFinal merged dataset shape: {merged.shape}')
print(merged['label'].value_counts(dropna=False))



Final merged dataset shape: (289, 4)
label
None    112
0       107
1        70
Name: count, dtype: int64


In [6]:
# --- Step 5: Save ---
merged.to_csv('merged_dataset.csv', index=False)
print('\nSaved to merged_dataset.csv')



Saved to merged_dataset.csv


In [9]:
#load the merged dataset to check
merged_check = pd.read_csv('merged_dataset.csv')
print(f'\nLoaded merged dataset shape: {merged_check.shape}')
print(merged_check['label'].value_counts(dropna=False))


Loaded merged dataset shape: (289, 4)
label
0    173
1    116
Name: count, dtype: int64


In [19]:
'''
Now I wanna add sleep tag in the dataset,
i will randomly pick 20 posts from posts_processed.csv 
big data file and merge it to the merged dataset and label 
them as relevant

'''
df3 = pd.read_csv('posts_processed2.csv')
# --- Step 6: Randomly sample 20 posts from df3 if keyword =sleep---
sleep_posts = df3[df3['text'].str.contains('sleep', case=False, na=False)]


sampled_sleep_posts = sleep_posts.sample(n=20, random_state=42)
# --- Step 7: Format sampled sleep posts to match merged dataset schema ---
sampled_sleep_formatted = pd.DataFrame({
    'id':               sampled_sleep_posts['id'],
    'text':             sampled_sleep_posts['text'],
    'similarity_score': None,  # No similarity score for these new posts
    'label':            1     # Label as relevant
})


# --- Step 8: Merge sampled sleep posts into the merged dataset ---
final_merged = pd.concat([merged_check, sampled_sleep_formatted], ignore_index=True)
print(f'\nFinal merged dataset shape after adding sleep posts: {final_merged.shape}')
print(final_merged['label'].value_counts(dropna=False))
# --- Step 9: Save the final merged dataset ---
final_merged.to_csv('final_merged_dataset.csv', index=False)
print('\nSaved to final_merged_dataset.csv')


Final merged dataset shape after adding sleep posts: (309, 4)
label
0    173
1    136
Name: count, dtype: int64

Saved to final_merged_dataset.csv


/var/folders/r1/_8ps3x2x6nb6r77rz8dp80sh0000gn/T/ipykernel_84952/2284816369.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_merged = pd.concat([merged_check, sampled_sleep_formatted], ignore_index=True)
